# Bayesian Inference in FSharp


## Introduction

For my F# Advent submission this year, I decided to implement an extremely simple Bayesian Statistical Framework to be used for inference. The purpose of statistical inference is to infer characteristics of a population via samples to be used for further tasks such as making predictions. The "Bayesian" prefix refers to making use of the Bayes Theorem to conduct inferential statistics.

![image.png](./Images/StatisticalInference.png)

I like to think of Statistical Inference via a pasta boiling analogy: hypothetically, assume you lose the cooking instructions of dried pasta you are trying to cook. You start boiling the water and don't know when the pasta is al-dente. One way to check is to careful remove a unit of it and bite into it. Through this process, you'll know right away if the pasta needs to cook more or is ready to be sauce'd up. Similarly, inference deals with trying to figure out characteristics (is the pasta ready?) of a population (all the pasta) via a sample (a single piece of pasta).

![image.png](./Images/pasta.jpeg)

## Motivation

I recently received a Masters in Data Science but discovered that none of my courses dove deep into Bayesian Statistics. Therefore, motivated by a penchant desire to fully understand the differences between frequentist approaches and Bayesian ones to eventually gain an understanding of Bayesian Neural Networks was the impetus behind trying to understand the statistical theory. 

While reading  ["A First Course in Bayesian Statistical Methods"](https://www.amazon.com/Bayesian-Statistical-Methods-Springer-Statistics/dp/1441928286/ref=sr_1_2?crid=3D7COZ06U9AU1&dchild=1&keywords=a+first+course+in+bayesian+statistical+methods&qid=1608705310&sprefix=A+first+course+in+baye%2Caps%2C239&sr=8-2) by Peter Hoff (highly recommend this book to those with a strong mathematical background), I discovered [pymc3](https://docs.pymc.io/), a Probabilistic Programming library in Python. Pymc3, in my opinion, has some great abstractions that I wanted to implement myself in a functional-first setting. 

The best way to learn a new statistical algorithm is my implementing it and the best way to learn something well is to teach it to others and hence, this submission is a result of that ideology. 

## Bayes Theorem

The crux of Bayesian Inference lies in making use of the Bayes Theorem whose formula is the following:

$p(\theta \mid y) = \frac{p(y \mid \theta) p(\theta)}{p(y)}$

| Term           | Formula | Statistical Definition  |
| :------------- | :----------: | -----------: |
|  Prior         |  $p(\theta)$  | Quantification of    |  |
| Likelihood     | $p(y \mid \theta)$ |  | | |
| Evidence       | $p(y)$ | Normalizing constant so that the probability is between 0 and 1 |
| Posterior      | $p(\theta \mid y)$ |  | | |

The premise of the interpretation of Bayes Theorem that resonates most with the scientific method is that you first come up with a hypothesis (prior), you figure out the likelihood that the hypothesis is valid based on available evidence and then quantify the validity of the hypothesis by highlighting the level of associated uncertainty. 


## Bayesian Inference in Practice

In practice, the denominator or the evidence i.e. the normalization constant of the Bayes theorem is a pain to calculate. For example, for continuous variables, Bayes theorem has the following form:

$p(\theta \mid y) = \frac{p(y \mid \theta) p(\theta)}{\int_{\Theta}p(y \mid \tilde{\theta}) p(\tilde{\theta}) \; d\tilde{\theta}}$

That integral is UGLY and sometimes computational infeasible and so, approximation techniques must be used to get to generate the posterior distribution : enter __Markov-Chain Monte Carlo (MCMC)__ methods.

### Markov-Chain Monte-Carlo Methods


## Goal

The goal of this submission is to develop the following:

1. A Bayesian Domain Specific Language (DSL) that a Statistician with no programming experience can make use of for the simplest inferential case i.e. computing the posterior for a Bayesian Network, a Directed Acylic Graph (DAG), with a single prior and likelihood.
2. A way to represent the Bayesian DSL.
3. An algorithm that makes use of the Bayesian DSL representation to approximate the posterior distribution.

## Getting Setup

The inferential logic will be requiring minimal dependencies. 

In [12]:
#!fsharp

// Dependencies
#r "nuget: XPlot.Plotly"
#r "nuget: MathNet.Numerics"
#r "nuget: Newtonsoft.Json"

// Imports 
open XPlot.Plotly
open MathNet.Numerics
open Newtonsoft.Json

## Parsing Logic

In [5]:
#!fsharp
open System

// A parsed Bayesian Model is a list of Parsed Random Variables.
type ParsedRandomVariable = 
    { Name             : string; 
      Conditionals     : string list; 
      Distribution     : string; 
      Parameters       : string list; 
      Observed         : string option }
type ParsedBayesianModel = ParsedRandomVariable list

// Format: RVName [|Conditionals] ~ Distribution(Parameters) [: observed] 
// [] -> optional
// NOTE: There can't be any spaces in the distribution parameters.
let parseLineOfModel (lineInModel : string) : ParsedRandomVariable = 
 
    // Helper fn to split the string based on a variety of type of delimiters.
    // Resultant type is a list of strings to feed in for the pattern matching.
    let splitToList (toSplit : string) (delimiters : obj) : string list = 
        let split = 
            match delimiters with
            | :? string        as s   -> toSplit.Split(s, StringSplitOptions.RemoveEmptyEntries) 
            | :? array<string> as arr -> toSplit.Split(arr, StringSplitOptions.RemoveEmptyEntries) 
            | :? array<char>   as arr -> toSplit.Split(arr, StringSplitOptions.RemoveEmptyEntries) 
            | _ -> failwithf "Splitting based on delimiters failed as it is neither a string nor an array of strings: Of Type: %A - %A" (delimiters.GetType()) toSplit
        
        Array.toList split

    match splitToList lineInModel " " with
    | nameAndConditionals :: "~" :: distributionParametersObserved ->
        // Get the name and conditionals.
        let splitNameAndConditionals = splitToList nameAndConditionals "|"
        let name = splitNameAndConditionals.[0]
        let conditionals = 
            match splitNameAndConditionals with 
            | name :: conditionals -> 
                if conditionals.Length > 0 then splitToList conditionals.[0] ","
                else []
            | _ -> failwithf "Pattern not found for RV Name and Conditionals - the format is: RVName|Condtionals: %A" splitNameAndConditionals

        let extractAndGetParameters (distributionNameAndParameters : string) : string * string list = 
            let splitDistributionAndParameters = splitToList distributionNameAndParameters [| "("; ")" |]
            (splitDistributionAndParameters.[0], splitToList splitDistributionAndParameters.[1] ",")
            
        match distributionParametersObserved with 

        // Case: Without Observations. Example: theta ~ Gamma(a,b)
        | distributionNameAndParameters when distributionNameAndParameters.Length = 1 ->
            let extractedDistributionAndParameters = extractAndGetParameters distributionNameAndParameters.[0]
            { Name             = name; 
              Conditionals     = conditionals; 
              Distribution     = (fst extractedDistributionAndParameters).ToLower();
              Observed         = None; 
              Parameters       = snd extractedDistributionAndParameters; }

        // Case: With Observations. Example: Y|theta ~ Poisson(theta) : observed
        | distributionNameAndParameters :: ":" :: observed ->
            let extractedDistributionAndParameters = extractAndGetParameters distributionNameAndParameters
            { Name             = name;
              Conditionals     = conditionals; 
              Distribution     = (fst extractedDistributionAndParameters).ToLower();
              Observed         = Some observed.Head; 
              Parameters       = snd extractedDistributionAndParameters; } 

        // Case: Error.
        | _ -> failwithf "Pattern not found for the model while parsing the distribution, parameters and optionally, the observed variables: %A" distributionParametersObserved 

    | _ -> failwithf "Pattern not found for the following line in the model - please check the syntax: %A" lineInModel

let parseModel (model : string) : ParsedBayesianModel = 
    model.Split('\n') 
    |> Array.toList
    |> List.map(parseLineOfModel)

let printParsedModel (model : string) : unit = 
    let parsedModel = parseModel model
    printfn "Model: %A is represented as %A" model parsedModel

In [6]:
#!fsharp
// Print out our simple 1-Parameter Model.
let model1 = @"theta ~ Gamma(a,b)
              Y|theta ~ Poisson(theta) : observed"
printParsedModel(model1)

let model2 = @"theta ~ Beta(unit,unit)
               Y|theta ~ Binomial(n,theta) : observed"
printParsedModel(model2)

// This model doesn't make sense but adding to test multiple conditionals.
let model3 = @"theta ~ Beta(unit,unit)
               gamma ~ Gamma(a,b)
               Y|theta,gammma ~ Binomial(n,theta) : observed"
printParsedModel(model3)

Model: "theta ~ Gamma(a,b)
              Y|theta ~ Poisson(theta) : observed" is represented as [{ Name = "theta"
   Conditionals = []
   Distribution = "gamma"
   Parameters = ["a"; "b"]
   Observed = None }; { Name = "Y"
                        Conditionals = ["theta"]
                        Distribution = "poisson"
                        Parameters = ["theta"]
                        Observed = Some "observed" }]
Model: "theta ~ Beta(unit,unit)
               Y|theta ~ Binomial(n,theta) : observed" is represented as [{ Name = "theta"
   Conditionals = []
   Distribution = "beta"
   Parameters = ["unit"; "unit"]
   Observed = None }; { Name = "Y"
                        Conditionals = ["theta"]
                        Distribution = "binomial"
                        Parameters = ["n"; "theta"]
                        Observed = Some "observed" }]
Model: "theta ~ Beta(unit,unit)
               gamma ~ Gamma(a,b)
               Y|theta,gammma ~ Binomial(n,theta) : observed" is repre

In [9]:
#!fsharp
open XPlot.Plotly
open MathNet.Numerics.Distributions

let x = 
    ContinuousUniform().Samples()
    |> Seq.take 10000
    |> Seq.toArray

Histogram(x = x) 
|> Chart.Plot
|> Chart.WithWidth 700
|> Chart.WithHeight 500

## Lessons and How I Can Improve The Implementation

## References

Mr Bungle's first album for motivation